# 0.0. Imports

In [26]:
!pip install umap-learn -q

In [29]:
!python3 -c "import umap.umap_ as umap; print('umap SDK version: {}'.format(umap.__version__))"

Traceback (most recent call last):
  File "<string>", line 1, in <module>
AttributeError: module 'umap.umap_' has no attribute '__version__'


In [31]:
!pip freeze

absl-py==1.4.0
aiohttp==3.8.4
aiohttp-cors==0.7.0
aiorwlock==1.3.0
aiosignal==1.3.1
ansiwrap==0.8.4
anyio @ file:///home/conda/feedstock_root/build_artifacts/anyio_1666191106763/work/dist
argon2-cffi @ file:///home/conda/feedstock_root/build_artifacts/argon2-cffi_1640817743617/work
argon2-cffi-bindings @ file:///home/conda/feedstock_root/build_artifacts/argon2-cffi-bindings_1649500320262/work
async-timeout==4.0.2
asynctest==0.13.0
attrs @ file:///home/conda/feedstock_root/build_artifacts/attrs_1671632566681/work
Babel==2.12.1
backcall @ file:///home/conda/feedstock_root/build_artifacts/backcall_1592338393461/work
backoff==2.2.1
backports.functools-lru-cache @ file:///home/conda/feedstock_root/build_artifacts/backports.functools_lru_cache_1618230623929/work
beatrix-jupyterlab @ file:///home/kbuilder/miniconda3/conda-bld/dlenv-base_1679880048420/work/packages/beatrix_jupyterlab-2023.324.164149.tar.gz
beautifulsoup4 @ file:///home/conda/feedstock_root/build_artifacts/beautifulsoup4_167932

In [63]:
import pandas as pd
import numpy as np
from typing import Tuple
from functools import reduce

from google.cloud import bigquery

import umap.umap_ as umap
from sklearn.preprocessing import MinMaxScaler
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import silhouette_score

# 1.0. Loading data

In [2]:
%%bigquery df_raw

SELECT * FROM `gcp-vertex.gcp_bq.dados_ecommerce_iterm`
WHERE InvoiceDate <= "2023-05-15"

Query is running:   0%|          |

Downloading:   0%|          |

In [3]:
df_raw.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,553185,22266,EASTER DECORATION HANGING BUNNY,14,2017-05-13,0.65,17504.0,United Kingdom
1,553185,22139,RETROSPOT TEA SET CERAMIC 11 PC,1,2017-05-13,4.95,17504.0,United Kingdom
2,553185,21792,CLASSIC FRENCH STYLE BASKET GREEN,4,2017-05-13,3.95,17504.0,United Kingdom
3,553185,23133,LARGE IVORY HEART WALL ORGANISER,4,2017-05-13,8.25,17504.0,United Kingdom
4,553185,23142,IVORY WIRE KITCHEN ORGANISER,4,2017-05-13,10.40,17504.0,United Kingdom


In [4]:
df_raw.shape

(404714, 8)

# 2.0. Data preparation

In [5]:
# trocas os tipos
# dois dataframes, compras e retornos

In [6]:
df_raw['CustomerID'] = df_raw['CustomerID'].astype(int)

In [7]:
df_raw['InvoiceDate'] = pd.to_datetime(df_raw['InvoiceDate'])

In [8]:
df_raw.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

In [9]:
def coluna_para_int(dataframe: pd.DataFrame, nome_coluna: str) -> bool:
    """
    Converte a coluna especificada em um dataframe para o tipo inteiro.

    Args:
        dataframe (pd.DataFrame): O dataframe a ser processado.
        nome_coluna (str): O nome da coluna a ser convertida.

    Returns:
        bool: True se a conversão foi bem sucedida, False caso contrário.
    """
    
    if nome_coluna not in dataframe.columns:
        raise ValueError(f'Coluna {nome_coluna} nao encontrada')
        
    try:
        dataframe[nome_coluna] = dataframe[nome_coluna].astype(int)
    except:
        raise ValueError(f'Nao foi possivel converter a coluna {nome_coluna} para inteiro.')
    return True

In [10]:
coluna_para_int(df_raw, 'CustomerID')

True

In [11]:
def coluna_para_date(dataframe: pd.DataFrame, nome_coluna: str, formato_data: str = None) -> bool: 
    """
    Descricao
    
    Args:
    
    Retornos:
    """
    
    if nome_coluna not in dataframe.columns:
        raise ValueError(f'Coluna {nome_coluna} nao foi encontrada no dataframe de entrada')

    try:
        if formato_data:
            dataframe[nome_coluna] = pd.to_datetime(dataframe[nome_coluna], format=formato_data)
        else:
            dataframe[nome_coluna] = pd.to_datetime(dataframe[nome_coluna])
    except (ValueError, TypeError):
        raise ValueError(f"Could not convert column '{nome_coluna}'")

    # Retorna True se a conversão foi bem sucedida
    return True

In [12]:
def keep_features(dataframe: pd.DataFrame, keep_columns: list) -> pd.DataFrame:
    """
    Retorna um DataFrame com as colunas especificadas em keep_columns.
    
    Args:
        dataframe (pd.DataFrame): O DataFrame a ser processado.
        keep_columns (list): A lista de nomes de colunas a serem mantidas no DataFrame resultante.

    Returns:
        pd.DataFrame: O DataFrame resultante com apenas as colunas especificadas em keep_columns.
    """
    return dataframe[keep_columns]

def filtragem_features(dataframe_raw: pd.DataFrame) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """
    Descricao
    
    Args:
    
    Retornos:
    """
    df_returns = dataframe_raw.loc[dataframe_raw['Quantity'] < 0, ['CustomerID', 
                                                                   'Quantity']]
    df_purchases = dataframe_raw.loc[dataframe_raw['Quantity'] >= 0, :]
    
    # Filter main data
    df_filtered = keep_features(dataframe_raw, ['InvoiceNo', 'StockCode', 'Quantity',
                                                'InvoiceDate', 'UnitPrice', 
                                                'CustomerID', 'Country'])
    
    return df_filtered, df_purchases, df_returns

In [13]:
df_filtered, df_purchases, df_returns = filtragem_features(df_raw)

In [14]:
df_filtered.head(1)

,InvoiceNo,StockCode,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,553185,22266,14,2017-05-13,0.65,17504,United Kingdom


In [15]:
df_purchases.head(1)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,553185,22266,EASTER DECORATION HANGING BUNNY,14,2017-05-13,0.65,17504,United Kingdom


In [16]:
df_returns.head(1)

,CustomerID,Quantity
761,12471,-3


# 3.0. Feature engineering

In [17]:
def calculate_gross_revenue(dataframe_purchases: pd.DataFrame) -> pd.DataFrame:
    """
    Calcula a receita bruta de cada cliente com base nas colunas 'Quantity' e 'UnitPrice' e retorna
    um DataFrame com as colunas 'CustomerID' e 'gross_revenue'.

    Args:
        dataframe_purchases (pd.DataFrame): O DataFrame das compras contendo as colunas 'CustomerID', 'Quantity' e 'UnitPrice'.

    Returns:
        pd.DataFrame: O DataFrame resultante contendo as colunas 'CustomerID' e 'gross_revenue'.
    """
    # Verifica se as colunas necessárias estão presentes no DataFrame de entrada
    required_columns = {'CustomerID', 'Quantity', 'UnitPrice'}
    missing_columns = required_columns - set(dataframe_purchases.columns)
    if missing_columns:
        raise ValueError(f"O DataFrame de entrada está faltando as seguintes colunas: {missing_columns}")
    
    # Calcula a receita bruta e agrupa por CustomerID
    df = dataframe_purchases.copy()
    df.loc[:, 'gross_revenue'] = df.loc[:, 'Quantity'] * df.loc[:, 'UnitPrice']
    grouped_df = df.groupby('CustomerID').agg({'gross_revenue': 'sum'}).reset_index().copy()
    
    return grouped_df

def create_recency(dataframe_purchases: pd.DataFrame, dataframe_filtered: pd.DataFrame) -> pd.DataFrame:
    """
    Calcula a recência da última compra para cada cliente.

    Args:
        dataframe_purchases (pd.DataFrame): DataFrame com as informações de compras de todos os clientes.
        dataframe_filtered (pd.DataFrame): DataFrame filtrado apenas com as informações dos clientes que desejamos calcular a recência.

    Returns:
        pd.DataFrame: DataFrame com as colunas 'CustomerID' e 'recency_days', indicando a recência em dias da última compra para cada cliente.

    """
    required_columns = {'CustomerID', 'InvoiceDate'}
    missing_columns = required_columns - set(dataframe_purchases.columns)
    if missing_columns:
        raise ValueError(f"O DataFrame de entrada está faltando as seguintes colunas: {missing_columns}")
    # calcula a data da última compra de cada cliente
    df_recency = dataframe_purchases.loc[:, ['CustomerID', 'InvoiceDate']].groupby('CustomerID').max().reset_index()
    
    # calcula a recência em dias da última compra de cada cliente em relação à data mais recente da base de dados filtrada
    df_recency['recency_days'] = (dataframe_filtered['InvoiceDate'].max() - df_recency['InvoiceDate']).dt.days
    
    # retorna o DataFrame apenas com as colunas 'CustomerID' e 'recency_days'
    return df_recency[['CustomerID', 'recency_days']]

def create_quantity_purchased(dataframe_purchases: pd.DataFrame) -> pd.DataFrame:
    """
    Calcula a quantidade de produtos adquiridos por cada cliente.

    Args:
        dataframe_purchases (pd.DataFrame): DataFrame com as informações de compras de todos os clientes.

    Returns:
        pd.DataFrame: DataFrame com as colunas 'CustomerID' e 'qty_products', indicando a quantidade de produtos adquiridos por cada cliente.
    """
    required_columns = {'CustomerID', 'StockCode'}
    missing_columns = required_columns - set(dataframe_purchases.columns)
    if missing_columns:
        raise ValueError(f"O DataFrame de entrada está faltando as seguintes colunas: {missing_columns}")
    # agrupa as informações de compras por CustomerID e conta o número de StockCode para cada grupo
    qty_purchased = dataframe_purchases.loc[:, ['CustomerID', 'StockCode']].groupby('CustomerID').count()
    
    # renomeia a coluna StockCode para qty_products e reseta o índice para transformar o CustomerID em uma coluna
    qty_purchased = qty_purchased.reset_index().rename(columns={'StockCode': 'qty_products'})
    
    # retorna o DataFrame com as colunas 'CustomerID' e 'qty_products'
    return qty_purchased

def create_freq_purchases(dataframe_purchases: pd.DataFrame) -> pd.DataFrame:
    """
    Calculates the purchase frequency of each customer based on the purchase history.

    Parameters
    ----------
    dataframe_purchases : pd.DataFrame
        DataFrame with purchase history of each customer, containing columns CustomerID, InvoiceNo, and InvoiceDate.

    Returns
    -------
    pd.DataFrame
        DataFrame with the purchase frequency of each customer, containing columns CustomerID and frequency.
    """
    required_columns = {'CustomerID', 'InvoiceNo', 'InvoiceDate'}
    missing_columns = required_columns - set(dataframe_purchases.columns)
    if missing_columns:
        raise ValueError(f"O DataFrame de entrada está faltando as seguintes colunas: {missing_columns}")

    # Calculate time range of purchases for each customer
    df_aux = (dataframe_purchases[['CustomerID', 'InvoiceNo', 'InvoiceDate']]
              .drop_duplicates()
              .groupby('CustomerID')
              .agg(max_=('InvoiceDate', 'max'),
                   min_=('InvoiceDate', 'min'),
                   days_=('InvoiceDate', lambda x: ((x.max() - x.min()).days) + 1),
                   buy_=('InvoiceNo', 'count'))
              .reset_index())

    # Calculate frequency of purchases for each customer
    df_aux['frequency'] = df_aux[['buy_', 'days_']].apply(
        lambda x: x['buy_'] / x['days_'] if x['days_'] != 0 else 0, axis=1)

    return df_aux

def create_qty_returns(dataframe_returns: pd.DataFrame) -> pd.DataFrame:
    """
    Computes the total quantity of returned products for each customer.

    Args:
        dataframe_returns: A pandas DataFrame containing information about returns.

    Returns:
        A pandas DataFrame with the total quantity of returned products for each customer.
    """
    # Validate input data
    
    if not all(col in dataframe_returns.columns for col in ['CustomerID', 'Quantity']):
        raise ValueError("Input DataFrame must contain 'CustomerID' and 'Quantity' columns")
    
    # Compute quantity of returns
    df_returns = dataframe_returns[['CustomerID', 'Quantity']].groupby('CustomerID').sum().reset_index().rename(columns={'Quantity': 'qty_returns'})
    df_returns['qty_returns'] = df_returns['qty_returns']* -1
    
    return df_returns

def run_feature_engineering(dataframe_filtered: pd.DataFrame, dataframe_purchases: pd.DataFrame, dataframe_returns: pd.DataFrame) -> pd.DataFrame:
    """
    Performs feature engineering on the input dataframes and returns a new dataframe with the engineered features.

    Args:
        dataframe_filtered: A pandas DataFrame containing filtered customer order data.
        dataframe_purchases: A pandas DataFrame containing customer purchase data.
        dataframe_returns: A pandas DataFrame containing customer return data.

    Returns:
        A pandas DataFrame with the engineered features for each customer.
    """
    # Check if input dataframes are empty
    if dataframe_filtered.empty:
        raise ValueError("Input DataFrame 'dataframe_filtered' is empty")
    if dataframe_purchases.empty:
        raise ValueError("Input DataFrame 'dataframe_purchases' is empty")

    # Check if required columns are present in input dataframes
    required_columns = ['CustomerID', 'InvoiceDate', 'StockCode', 'Quantity', 'UnitPrice']
    for df, name in zip([dataframe_filtered, dataframe_purchases], ['dataframe_filtered', 'dataframe_purchases']):
        missing_columns = set(required_columns) - set(df.columns)
        if missing_columns:
            raise ValueError(f"Missing columns {missing_columns} in input DataFrame '{name}'")
    if 'CustomerID' not in dataframe_returns.columns:
        raise ValueError("Column 'CustomerID' not found in input DataFrame 'dataframe_returns'")
    if 'Quantity' not in dataframe_returns.columns:
        raise ValueError("Column 'Quantity' not found in input DataFrame 'dataframe_returns'")

    # Perform feature engineering
    df_fengi = keep_features(dataframe_filtered, ['CustomerID']).drop_duplicates(ignore_index=True)
    gross_revenue = calculate_gross_revenue(dataframe_purchases)
    df_recency = create_recency(dataframe_purchases, dataframe_filtered)
    df_qty_products = create_quantity_purchased(dataframe_purchases)
    df_freq = create_freq_purchases(dataframe_purchases)
    returns = create_qty_returns(dataframe_returns)

    # Merge dataframes
    dfs = [df_fengi, gross_revenue, df_recency, df_qty_products, df_freq, returns]
    df_fengi = reduce(lambda left,right: pd.merge(left, right, on='CustomerID', how='left'), dfs)

    # Fill NaN values
    df_fengi['qty_returns'] = df_fengi['qty_returns'].fillna(0)

    # Select final features and return dataframe
    features = ['CustomerID', 'gross_revenue', 'recency_days', 'qty_products', 'frequency', 'qty_returns']
    return keep_features(df_fengi, features).dropna()

In [18]:
df_fengi = run_feature_engineering(df_filtered, df_purchases, df_returns)
df_fengi.head()

,CustomerID,gross_revenue,recency_days,qty_products,frequency,qty_returns
0,17504,2997.03,206.0,127.0,0.076923,208
1,15440,383.68,54.0,62.0,0.019355,0
2,16034,437.7,196.0,22.0,0.021898,0
3,16076,2234.73,3.0,129.0,0.048544,72
4,13094,1927.32,21.0,24.0,0.034483,174


In [19]:
df_fengi.shape

(4329, 6)

# 4.0. EDA


In [21]:
df_eda = keep_features(df_fengi, ['gross_revenue', 'recency_days', 'qty_products', 'frequency', 'qty_returns'])
df_eda.head()

,gross_revenue,recency_days,qty_products,frequency,qty_returns
0,2997.03,206.0,127.0,0.076923,208
1,383.68,54.0,62.0,0.019355,0
2,437.7,196.0,22.0,0.021898,0
3,2234.73,3.0,129.0,0.048544,72
4,1927.32,21.0,24.0,0.034483,174


# 5.0. Experimentacao

## 5.1. K clusters

In [25]:
k_clusters = np.arange(2, 5, 1)
k_clusters

array([2, 3, 4])

## 5.2. Scaler

In [32]:
scaler = MinMaxScaler()

## 5.3. Algoritmos de reducao de dimensionalidade

In [33]:
def train_pca(
    scaler, dataframe: pd.DataFrame, to_scaling: bool = True, **kwargs
) -> pd.DataFrame:
    """
    Treina um modelo PCA no dataframe de entrada e aplica-o aos dados usando o scaler especificado.

    Args:
        scaler: objeto scaler para transformar os dados
        dataframe (pd.DataFrame): dataframe de entrada
        to_scaling (bool): indica se os dados devem ser escalados antes de aplicar o PCA (padrão True)
        **kwargs: argumentos adicionais para a inicialização do modelo PCA
    Raises:
        ValueError: se o dataframe de entrada estiver vazio
    """
    if dataframe.empty:
        raise ValueError("Input dataframe is empty.")

    if to_scaling:
        X = scaler.fit_transform(dataframe)
    else:
        X = dataframe.copy()
    pca = PCA(**kwargs)
    principal_components = pca.fit_transform(X)
    df_pca = pd.DataFrame(principal_components)
    return df_pca


def train_umap(
    scaler, dataframe: pd.DataFrame, to_scaling: bool = True, **kwargs
) -> pd.DataFrame:
    """ """
    if dataframe.empty:
        raise ValueError("Input dataframe is empty.")

    if to_scaling:
        X = scaler.fit_transform(dataframe)
    else:
        X = dataframe.copy()

    reducer = umap.UMAP(random_state=42, **kwargs)
    embedding = reducer.fit_transform(X)

    df_umap = pd.DataFrame()
    df_umap["embedding_x"] = embedding[:, 0]
    df_umap["embedding_y"] = embedding[:, 1]
    return df_umap


def train_tsne(
    scaler, dataframe: pd.DataFrame, to_scaling: bool = True, **kwargs
) -> pd.DataFrame:
    """ """
    if dataframe.empty:
        raise ValueError("Input dataframe is empty.")

    if to_scaling:
        X = scaler.fit_transform(dataframe)
    else:
        X = dataframe.copy()

    reducer = TSNE(random_state=42, **kwargs)
    embedding = reducer.fit_transform(X)

    df_tsne = pd.DataFrame()
    df_tsne["embedding_x"] = embedding[:, 0]
    df_tsne["embedding_y"] = embedding[:, 1]
    return df_tsne

def train_tree_embedding(scaler, dataframe: pd.DataFrame,
    target: str = "gross_revenue", **kwargs
) -> pd.DataFrame:
    """
    
    """
    
    # Separate features and target variable
    X = dataframe.drop(columns=[target], axis=1)
    X = pd.DataFrame(scaler.fit_transform(X.values), 
                     columns=X.columns, 
                     index=X.index)
    y = dataframe[target]
    
    # Define and train the model
    rf_model = RandomForestRegressor(random_state=42, **kwargs)
    rf_model.fit(X, y)

    # Compute leaf indices for each sample
    leaf_indices = rf_model.apply(X)

    # Create DataFrame with leaf indices
    df_leaf = pd.DataFrame(leaf_indices)

    return df_leaf

In [34]:
df_pca = train_pca(scaler, df_eda, n_components=2)

In [35]:
df_umap = train_umap(scaler, df_eda)

In [36]:
df_tsne = train_tsne(scaler, df_eda)

/opt/conda/lib/python3.7/site-packages/sklearn/manifold/_t_sne.py:783: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  FutureWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/manifold/_t_sne.py:793: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  FutureWarning,


In [37]:
df_tree = train_tree_embedding(scaler, df_eda)

In [41]:
df_tree.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,4767,4784,4760,5226,4612,5050,5197,5137,4787,5202,...,5063,4628,4786,4616,4543,4790,4564,4553,4808,5083
1,1446,1475,1501,3361,1166,3322,1474,1557,1143,945,...,1119,1003,1160,1010,1294,1160,1016,1057,1453,1269
2,688,925,942,651,660,579,434,428,419,1376,...,322,455,411,425,895,490,405,424,980,622
3,4608,4712,4725,5200,4593,4995,5157,5109,4703,5215,...,5050,4473,3728,4561,4388,4779,4535,4448,4469,5051
4,1703,1089,822,1144,707,702,2111,722,1624,2186,...,2202,1566,580,1480,1055,1563,1533,506,1984,1857


## 5.4. Algoritmos de clusterização

In [42]:
# gmm e kmeans

def train_kmeans(dataframe: pd.DataFrame, k_clusters: list, **kwargs) -> list:
    """
    Treina o modelo KMenas para diferentes valores de k_clusters e calcula a pontuação de silhueta para cada um.

    Args:
        dataframe: um dataframe com as features de treinamento
        k_clusters: uma lista de inteiros que representa o número de clusters a serem testados
        **kwargs: argumentos adicionais para a inicialização do modelo de mistura gaussiana

    Returns:
        Uma lista de pontuações de silhueta correspondentes a cada valor de k_clusters.
    """

    silhouette_scores = []

    for k in k_clusters:
        # Model training
        kmeans_model = KMeans(n_clusters=k, random_state=42, **kwargs).fit(dataframe)

        # Model predict
        labels = kmeans_model.predict(dataframe)

        # Model performance
        silhouette = silhouette_score(dataframe, labels, metric="euclidean")
        silhouette_scores.append(silhouette)

    return silhouette_scores

def train_gmm(dataframe: pd.DataFrame, k_clusters: list, **kwargs) -> list:
    """
    
    """

    silhouette_scores = []

    for k in k_clusters:
        # Model training
        gmm_model = GaussianMixture(n_components=k, random_state=42, **kwargs).fit(dataframe)

        # Model predict
        labels = gmm_model.predict(dataframe)

        # Model performance
        silhouette = silhouette_score(dataframe, labels, metric="euclidean")
        silhouette_scores.append(silhouette)

    return silhouette_scores

In [59]:
kmeans_list = train_kmeans(df_pca, k_clusters, n_init=10)

In [60]:
gmm_list = train_gmm(df_pca, k_clusters, n_init=100)

In [47]:
k_clusters

array([2, 3, 4])

In [50]:
str(scaler)

'MinMaxScaler()'

In [61]:
df_results_pca = pd.DataFrame({
                                'scaler': str(scaler),
                                'Reducer': 'PCA',
                                'Clusters': k_clusters,
                                'KMeans': kmeans_list,
                                'GMM': gmm_list,
                                })
df_results_pca

,scaler,Reducer,Clusters,KMeans,GMM
0,MinMaxScaler(),PCA,2,0.729026,-0.116664
1,MinMaxScaler(),PCA,3,0.655202,0.486397
2,MinMaxScaler(),PCA,4,0.579898,0.352297


In [54]:
gmm_list = train_gmm(df_umap, k_clusters, n_init=10)
kmeans_list = train_kmeans(df_umap, k_clusters, n_init=100)

results_umap = pd.DataFrame({    'scaler': str(scaler),
                  'Reducer': 'UMAP',
                  'Clusters': k_clusters,
                  'GMM': gmm_list,
                  'KMeans': kmeans_list})
results_umap

,scaler,Reducer,Clusters,GMM,KMeans
0,MinMaxScaler(),UMAP,2,0.372851,0.405647
1,MinMaxScaler(),UMAP,3,0.478889,0.478889
2,MinMaxScaler(),UMAP,4,0.401634,0.455504


In [55]:
df_leaf = train_tree_embedding(scaler, df_eda, n_jobs=-1, n_estimators=100)

df_tree_tsne = train_tsne(scaler, df_leaf, to_scaling=False)
gmm_list_tree_tsne = train_gmm(df_tree_tsne, k_clusters, n_init=100)
kmeans_list_tree_tsne = train_kmeans(df_tree_tsne, k_clusters, n_init=100)
df_results_tree_tsne = pd.DataFrame( {
                  'scaler': str(scaler),
                  'Reducer': 'Tree Embedding + t-SNE',
                  'Clusters': k_clusters,
                  'GMM': gmm_list_tree_tsne,
                  'KMeans': kmeans_list_tree_tsne})

/opt/conda/lib/python3.7/site-packages/sklearn/manifold/_t_sne.py:783: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  FutureWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/manifold/_t_sne.py:793: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  FutureWarning,


In [56]:
df_results_tree_tsne

,scaler,Reducer,Clusters,GMM,KMeans
0,MinMaxScaler(),Tree Embedding + t-SNE,2,0.351803,0.350242
1,MinMaxScaler(),Tree Embedding + t-SNE,3,0.356614,0.381196
2,MinMaxScaler(),Tree Embedding + t-SNE,4,0.379406,0.400104


In [57]:
df_tree_pca = train_pca(scaler, df_leaf, to_scaling=False)
gmm_list_tree_pca = train_gmm(df_tree_pca, k_clusters, n_init=100)
kmeans_list_tree_pca = train_kmeans(df_tree_pca, k_clusters, n_init=100)
df_results_tree_pca = pd.DataFrame( {
                  'scaler': str(scaler),
                  'Reducer': 'Tree Embedding + PCA',
                  'Clusters': k_clusters,
                  'GMM': gmm_list_tree_pca,
                  'KMeans': kmeans_list_tree_pca})
df_results_tree_pca

,scaler,Reducer,Clusters,GMM,KMeans
0,MinMaxScaler(),Tree Embedding + PCA,2,0.420825,0.487065
1,MinMaxScaler(),Tree Embedding + PCA,3,0.483154,0.490610
2,MinMaxScaler(),Tree Embedding + PCA,4,0.518606,0.433822


In [62]:
df_final_results = pd.concat([df_results_pca, results_umap, df_results_tree_tsne, df_results_tree_pca], axis=0)
df_final_results

,scaler,Reducer,Clusters,KMeans,GMM
0,MinMaxScaler(),PCA,2,0.729026,-0.116664
1,MinMaxScaler(),PCA,3,0.655202,0.486397
2,MinMaxScaler(),PCA,4,0.579898,0.352297
0,MinMaxScaler(),UMAP,2,0.405647,0.372851
1,MinMaxScaler(),UMAP,3,0.478889,0.478889
2,MinMaxScaler(),UMAP,4,0.455504,0.401634
0,MinMaxScaler(),Tree Embedding + t-SNE,2,0.350242,0.351803
1,MinMaxScaler(),Tree Embedding + t-SNE,3,0.381196,0.356614
2,MinMaxScaler(),Tree Embedding + t-SNE,4,0.400104,0.379406
0,MinMaxScaler(),Tree Embedding + PCA,2,0.487065,0.420825


## 5.6. Salvando os resultados

In [64]:
def salvar_no_bigquery(dataframe: pd.DataFrame,
                      project_name: str,
                      dataset_table_name: str):
    client = bigquery.Client(project=project_name)
    
    job = client.load_table_from_dataframe(dataframe, dataset_table_name)
    job.result()

In [65]:
salvar_no_bigquery(df_final_results,
                  project_name='gcp-vertex',
                  dataset_table_name='gcp_bq.resultado_experimentos_clustering_2')